Vul hieronder de gegevens van de toets in.

In [ ]:
bestandslocatie_resultaten = 'Resultaten tentamen - anoniem.xlsx'

points = {
    "1a": 1,
    "1b": 1,
    "2a": 1,
    "2b": 1,
    "3": 1,
    "4": 1,
    "5a": 1,
    "5b": 1,
    "6": 1,
    "7": 1,
    "8a": 1,
    "8b": 1,
    "9": 1,
    "10a": 1,
    "10b": 1,
    "11": 1,
    "12a": 1,
    "12b": 1
}

Run onderstaande cel om de benodigde functie te defeniëren.

In [ ]:
import pandas as pd
import numpy as np

def analyseer_toets(df, points, is_original = True):
    df = df[df.columns.drop(list(df.filter(regex='Unnamed')))] # Verwijder lege kolommen
    df.columns = [str(name) for name in df.columns] # Kolomtitels omzetten naar string

    total_points = np.sum([value for _, value in points.items()]) # Totaal aantal punten op tentamen

    df = df.round({'Cijfer':1}) # Cijfer afronden op 1 decimaal

    # Start checks
    print(f'Het totale aantal punten van het tentamen is als het goed is {total_points}. Klopt dit?\n')
    if is_original: # Deze check alleen doen als er geen vragen geschrapt zijn (dit doe je altijd als eerste stap)
        for index, row in df.iterrows(): # Check of studenten bij vragen niet meer punten gekregen hebben dan mogelijk is
            for exercise, point_total in points.items():
                if row[str(exercise)] > point_total:
                    print(f'Bij student "{row["Voornaam/tussenvoegsel"]} {row["Achternaam"]}" zijn bij vraag {exercise} meer punten toegekend dan toegestaan. Het maximale aantal punten is namelijk {point_total}, maar er zijn {row[str(exercise)]} punten toegekend.')
        print('')
        for index, row in df.iterrows(): # Check of cijfer in Excel klopt
                current_points = 0
                for exercise, point_total in points.items():
                    if not np.isnan(row[exercise]):
                        current_points += row[exercise]
                grade = np.round(current_points/total_points*9+1,1)

                if row['Cijfer'] != grade:
                    print(f'Bij student "{row["Voornaam/tussenvoegsel"]} {row["Achternaam"]}" staat het cijfer onjuist in Excel. Het cijfer dat in Excel staat is {row["Cijfer"]}, maar het cijfer moet {grade} zijn.')
        print('\n')
    else:
        for index, row in df.iterrows(): # Indien er vragen geschrapt zijn, moet het cijfer opnieuw berekend worden
                current_points = 0
                for exercise, point_total in points.items():
                    if not np.isnan(row[exercise]):
                        current_points += row[exercise]
                df.loc[index,'Cijfer'] = np.round(current_points/total_points*9+1,1)
        print('\n')

    # Start analyse
    n_studenten_ingeschreven = df.shape[0] # Aantal studenten die zich hebben ingeschreven in Osiris

    indices_to_delete = []
    for index, row in df.iterrows(): # Maak dataframe met enkel studenten die meer dan 0 punten hebben
        current_points = 0
        student_has_points = False
        for exercise, point_total in points.items():
            if not np.isnan(row[exercise]):
                student_has_points = True
                break
        if not student_has_points:
            indices_to_delete.append(index)

    df_met_punten = df.drop(index=indices_to_delete)

    n_studenten_met_punten = df_met_punten.shape[0] # Aantal studenten met meer dan 0 punten
    n_voldoendes = len(df[df['Cijfer']>=5.5])

    print(f"In totaal hebben {n_studenten_ingeschreven} studenten zich ingeschreven.")
    print(f"In totaal hebben {n_studenten_met_punten} studenten punten behaald op het tentamen/deelgenomen.\n")

    if n_studenten_met_punten < 50:
        print("LET OP: minder dan 50 studenten heeft deelgenomen aan het tentamen. Dit is eigenlijk te weinig om conclusies te kunnen trekken op basis van deze psychometrische analyse.\n")

    print(f"{np.round(n_voldoendes/n_studenten_met_punten*100,2)}% van de studenten die het tentamen hebben gemaakt, hebben een voldoende behaald.")
    print(f"{np.round(n_voldoendes/n_studenten_ingeschreven*100,2)}% van de studenten die zich voor het vak hebben ingeschreven, hebben een voldoende behaald.\n")
    print(f"Het hoogste behaalde cijfer is een {df_met_punten['Cijfer'].max()}.")
    print(f"Het laagste behaalde cijfer (boven de 1.0) is een {df_met_punten['Cijfer'].min()}.\n")
    print(f"Het gemiddelde cijfer (zonder enen) is een {round(df_met_punten['Cijfer'].mean(),1)}.")
    print(f"De modus is een {list(df_met_punten['Cijfer'].mode())}.")
    print(f"De mediaan is een {df_met_punten['Cijfer'].median()}.\n")

    # Bereken Cronbach's Alpha
    item_variances = df_met_punten[list(points.keys())].var(ddof=1)
    total_variance = df_met_punten[list(points.keys())].sum(axis=1).var(ddof=1)
    cronbach_alpha = (len(points) / (len(points) - 1)) * (1 - (item_variances.sum() / total_variance))
    cronbach_alpha_text = f"De Cronbach's Alpha voor dit tentamen is {round(cronbach_alpha,2)}."
    if cronbach_alpha < 0.70:
        print(cronbach_alpha_text + " Dit betekent dat de toets van slechte/middelmatige kwaliteit was.")
    elif cronbach_alpha < 0.80:
        print(cronbach_alpha_text + " Dit betekent dat de toets van middelmatige/voldoende kwaliteit was.")
    elif cronbach_alpha < 0.90:
        print(cronbach_alpha_text + " Dit betekent dat de toets van voldoende/goede kwaliteit was.")
    else:
        print(cronbach_alpha_text + " Dit betekent dat de toets van goede/zeer goede kwaliteit was.")

    # Bepaal statistieken van vragen
    questions = pd.DataFrame({'Vraag': pd.Series(dtype='string'),
                            'P-waarde': pd.Series(dtype='float64'),
                            'Rit': pd.Series(dtype='float64'),
                            'Rir': pd.Series(dtype='float64'),
                            'Conclusie': pd.Series(dtype='string')})
    for exercise, point_total in points.items():
        p_value = round(df_met_punten[exercise].sum()/(point_total*n_studenten_met_punten),2)
        rit = round(df_met_punten[exercise].corr(df_met_punten['Cijfer'], method='pearson'),2)

        df_rir = df_met_punten.drop(exercise,axis=1) # Dataframe zonder deze vraag, voor bepalen rir waarde

        for index, row in df_rir.iterrows(): # Cijfers bepalen als de vraag er niet in zou zitten
            current_points = 0
            for exercise_rir, point_total_rir in points.items():
                if not exercise == exercise_rir and not np.isnan(row[exercise_rir]):
                    current_points += row[exercise_rir]

            df_rir.loc[index,'Cijfer'] = np.round(current_points/(total_points-point_total)*9+1,1)

        rir = round(df_met_punten[exercise].corr(df_rir['Cijfer'], method='pearson'),2)

        conclusion_text = ""
        if p_value < 0.25:
            conclusion_text = 'Te moeilijk, '
        elif p_value > 0.9:
            conclusion_text = 'Te makkelijk, '
        if rir < 0.15:
            conclusion_text += "Kwaliteit: middelmatig/slecht"
        elif rir < 0.25:
            conclusion_text += "Kwaliteit: middelmatig/voldoende"
        elif rir < 0.35:
            conclusion_text += "Kwaliteit: voldoende/goed"
        else:
            conclusion_text += "Kwaliteit: goed/zeer goed"

        new_row = pd.DataFrame([[exercise,p_value,rit,rir,conclusion_text]],columns=['Vraag','P-waarde','Rit','Rir','Conclusie'])
        questions = pd.concat([questions,new_row])

    print('\nScores per vraag:')
    display(questions)

    klassen = df_met_punten.groupby('Klas')['Cijfer']

    gemiddelde_per_klas = klassen.mean().round(1).rename('Gemiddelde')
    n_deelnemers_per_klas = klassen.count().rename('# deelnemers')
    n_voldoendes_per_klas = klassen.apply(lambda x: (x >= 5.5).sum()).rename('# voldoendes')
    n_onvoldoendes_per_klas = klassen.apply(lambda x: (x < 5.5).sum()).rename('# onvoldoendes')

    resultaten_per_klas = pd.concat([gemiddelde_per_klas,n_deelnemers_per_klas,n_voldoendes_per_klas,n_onvoldoendes_per_klas],axis = 1)
    resultaten_per_klas['% voldoendes'] = pd.to_numeric(round(resultaten_per_klas['# voldoendes']/(resultaten_per_klas['# deelnemers'])*100), downcast='integer')
    print('\nResultaten per klas:')
    display(resultaten_per_klas)

    studios = df_met_punten.groupby('Studio')['Cijfer']

    gemiddelde_per_studio = studios.mean().round(1).rename('Gemiddelde')
    n_deelnemers_per_studio = studios.count().rename('# deelnemers')
    n_voldoendes_per_studio = studios.apply(lambda x: (x >= 5.5).sum()).rename('# voldoendes')
    n_onvoldoendes_per_studio = studios.apply(lambda x: (x < 5.5).sum()).rename('# onvoldoendes')

    resultaten_per_studio = pd.concat([gemiddelde_per_studio,n_deelnemers_per_studio,n_voldoendes_per_studio,n_onvoldoendes_per_studio],axis = 1)
    resultaten_per_studio['% voldoendes'] = pd.to_numeric(round(resultaten_per_studio['# voldoendes']/(resultaten_per_studio['# deelnemers'])*100), downcast='integer')
    print('\nResultaten per studio:')
    display(resultaten_per_studio)


Run onderstaande cel. Check bij de uitput of er geen fouten zijn gevonden. Als dit zo is, kun je de toets analyseren.

In [ ]:
df = pd.read_excel(bestandslocatie_resultaten,header=1)

analyseer_toets(df, points)

Als je eventueel een bepaalde vraag wilt weghalen om te kijken wat het effect hiervan is, kun je deze weghalen uit de 'points' dictionary. Zie voorbeeld hieronder.

In [ ]:
# remove question 1 from points.
points_without_one = points.copy()
points_without_one.pop('1')

analyseer_toets(df, points_without_one, False)